In [1]:
!pip install scanpy scipy umap-learn leidenalg
!pip install hmmlearn
!pip install scikit-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 12.7 MB/s eta 0:00:00


Set up

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import scipy.sparse
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_recall_fscore_support
from hmmlearn import hmm
from scipy import stats
from scipy.ndimage import uniform_filter1d
from scipy.ndimage import gaussian_filter1d
from scipy import stats
from scipy.signal import medfilt, savgol_filter
import re
import itertools

Fetch data

In [4]:
import scanpy as sc

adata = sc.read_10x_h5("/content/GSM8157262_9704-BC-2_48hr_filtered_feature_bc_matrix.h5")
adata.var_names_make_unique()


/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [5]:
# normalization + log1p
if adata.X.max() > 100:
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

# PCA
sc.pp.pca(adata)

# build neigbors
sc.pp.neighbors(adata)

# visualization（UMAP）
sc.tl.umap(adata)

# Leiden cluster
sc.tl.leiden(adata, key_added='leiden_clusters')

<ipython-input-5-2090faa52d8a>:16: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(adata, key_added='leiden_clusters')


Data Annotation

In [6]:
!pip install mygene

from mygene import MyGeneInfo
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.1 MB/s eta 0:00:00


In [7]:
mg = MyGeneInfo()

# get all gene names（default adata.var_names）
genes = adata.var_names.tolist()

# search for chromosome
query = mg.querymany(genes, scopes='symbol', fields='chromosome', species='human')


INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:querying 3001-4000 ...
INFO:biothings.client:querying 4001-5000 ...
INFO:biothings.client:querying 5001-6000 ...
INFO:biothings.client:querying 6001-7000 ...
INFO:biothings.client:querying 7001-8000 ...
INFO:biothings.client:querying 8001-9000 ...
INFO:biothings.client:querying 9001-10000 ...
INFO:biothings.client:querying 10001-11000 ...
INFO:biothings.client:querying 11001-12000 ...
INFO:biothings.client:querying 12001-13000 ...
INFO:biothings.client:querying 13001-14000 ...
INFO:biothings.client:querying 14001-15000 ...
INFO:biothings.client:querying 15001-16000 ...
INFO:biothings.client:querying 16001-17000 ...
INFO:biothings.client:querying 17001-18000 ...
INFO:biothings.client:querying 18001-18082 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing quer

In [8]:
from mygene import MyGeneInfo
mg = MyGeneInfo()

# insert adata.var_names as query
query = mg.querymany(
    adata.var_names.tolist(),
    scopes='symbol',
    fields='symbol,name,chromosome,genomic_pos',
    species='human'
)

# transform DataFrame and set index
df = pd.DataFrame(query).set_index('query')

# print out and check 'chromosome'、'genomic_pos'
print(df.columns)
print(df.head())

INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:querying 3001-4000 ...
INFO:biothings.client:querying 4001-5000 ...
INFO:biothings.client:querying 5001-6000 ...
INFO:biothings.client:querying 6001-7000 ...
INFO:biothings.client:querying 7001-8000 ...
INFO:biothings.client:querying 8001-9000 ...
INFO:biothings.client:querying 9001-10000 ...
INFO:biothings.client:querying 10001-11000 ...
INFO:biothings.client:querying 11001-12000 ...
INFO:biothings.client:querying 12001-13000 ...
INFO:biothings.client:querying 13001-14000 ...
INFO:biothings.client:querying 14001-15000 ...
INFO:biothings.client:querying 15001-16000 ...
INFO:biothings.client:querying 16001-17000 ...
INFO:biothings.client:querying 17001-18000 ...
INFO:biothings.client:querying 18001-18082 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing quer

Index(['_id', '_score', 'genomic_pos', 'name', 'symbol', 'notfound'], dtype='object')
            _id     _score                                        genomic_pos  \
query                                                                           
SAMD11   148398  17.714540  {'chr': '1', 'end': 944575, 'ensemblgene': 'EN...   
NOC2L     26155  17.598486  {'chr': '1', 'end': 959309, 'ensemblgene': 'EN...   
KLHL17   339451  17.669617  {'chr': '1', 'end': 965719, 'ensemblgene': 'EN...   
PLEKHN1   84069  17.698200  {'chr': '1', 'end': 975865, 'ensemblgene': 'EN...   
PERM1     84808  18.462875  {'chr': '1', 'end': 982117, 'ensemblgene': 'EN...   

                                                      name   symbol notfound  
query                                                                         
SAMD11            sterile alpha motif domain containing 11   SAMD11      NaN  
NOC2L    NOC2 like nucleolar associated transcriptional...    NOC2L      NaN  
KLHL17                        

In [9]:
# process genomic_pos
def unpack_genomic_pos(row):
    pos = row.get('genomic_pos')
    if isinstance(pos, dict):
        return pd.Series({
            'chromosome': pos.get('chr'),
            'start': pos.get('start'),
            'end': pos.get('end')
        })
    else:
        return pd.Series({'chromosome': None, 'start': None, 'end': None})

# apply to dataframe
df_pos = df.apply(unpack_genomic_pos, axis=1)

# avoid duplicate index
df_pos_dedup = df_pos.loc[~df_pos.index.duplicated(keep='first')]

# align index
adata.var['chromosome'] = df_pos_dedup['chromosome'].reindex(adata.var_names)
adata.var['start'] = df_pos_dedup['start'].reindex(adata.var_names)
adata.var['end'] = df_pos_dedup['end'].reindex(adata.var_names)

Functions

In [10]:
def detect_cnas_per_cluster(
    adata,
    cluster_key: str,
    cluster_refs: dict[str, list[str]],
    window_size: int = 100,
    min_genes_per_window: int = 10,
    smoothing_sigma: float = 1.0,
    lfc_threshold: float = 0.1,
    fraction_threshold: float = 0.2,
    layer: str | None = None
) -> tuple[pd.DataFrame, list[dict], pd.DataFrame]:
    """
    For each cluster in adata.obs[cluster_key], uses cluster_refs to compute
    smoothed per-window log2 fold-change and calls gains/losses by requiring
    that at least `fraction_threshold` of windows on a chromosome exceed
    +lfc_threshold (gain) or -lfc_threshold (loss).

    Returns
    -------
    calls_chr : pd.DataFrame
        DataFrame of shape (n_cells, n_chromosomes) with values in {-1, 0, +1}.
    """
    X = adata.layers[layer] if layer else adata.X
    if scipy.sparse.issparse(X):
        X = X.toarray()

    var = adata.var.copy()
    var['chrom'] = var['chromosome'].astype(str)
    var['start'] = pd.to_numeric(var['start'], errors='coerce')
    var = var.dropna(subset=['chrom','start'])
    def chr_key(c):
        c = str(c)
        return (int(c), '') if c.isdigit() else (np.inf, c)
    var['_idx'] = np.arange(var.shape[0])
    var = var.sort_values(['chrom','start'], key=lambda col: col.map(chr_key))
    sorted_idx = var['_idx'].values.astype(int)
    gene_chroms = var['chrom'].values

    Xs = X[:, sorted_idx]
    n_cells, n_win = Xs.shape

    windows = []
    window_expr = []
    for chrom in pd.unique(gene_chroms):
        idx_chr = np.where(gene_chroms == chrom)[0]
        step = max(1, window_size // 2)
        for i in range(0, len(idx_chr), step):
            block = idx_chr[i:i+window_size]
            if len(block) < min_genes_per_window:
                continue
            windows.append({'chrom': chrom, 'genes': block})
            window_expr.append(Xs[:, block].mean(axis=1))
    W = np.stack(window_expr, axis=1)

    chroms = sorted({w['chrom'] for w in windows}, key=chr_key)
    calls_chr = pd.DataFrame(0, index=adata.obs_names, columns=chroms, dtype=int)

    for cl in adata.obs[cluster_key].astype(str).unique():
        refs = cluster_refs.get(cl, [])
        if len(refs) < min_genes_per_window:
            continue
        mask_cl = (adata.obs[cluster_key].astype(str) == cl).values
        idx_cl = np.where(mask_cl)[0]
        mask_ref = adata.obs_names.isin(refs)
        idx_ref = np.where(mask_ref)[0]
        if len(idx_ref) == 0:
            continue
        ref_med = np.median(W[idx_ref, :], axis=0)
        ref_med = np.clip(ref_med, 1e-6, None)
        L = np.log2((W[idx_cl, :] + 1e-6) / ref_med[np.newaxis, :])
        if smoothing_sigma > 0:
            for chrom in chroms:
                win_idxs = [i for i,w in enumerate(windows) if w['chrom']==chrom]
                if len(win_idxs) > 1:
                    L[:, win_idxs] = gaussian_filter1d(
                        L[:, win_idxs],
                        sigma=smoothing_sigma,
                        axis=1,
                        mode='reflect'
                    )
        for chrom in chroms:
            win_idxs = [i for i,w in enumerate(windows) if w['chrom']==chrom]
            sub = L[:, win_idxs]
            frac_gain = (sub > lfc_threshold).sum(axis=1) / len(win_idxs)
            frac_loss = (sub < -lfc_threshold).sum(axis=1) / len(win_idxs)
            cells = adata.obs_names[idx_cl]
            calls_chr.loc[cells[frac_gain >= fraction_threshold], chrom] = +1
            calls_chr.loc[cells[frac_loss >= fraction_threshold], chrom] = -1

    return calls_chr, windows, var

In [11]:
def extract_cna_segments_fast(
    calls_chr: pd.DataFrame,
    windows: list[dict],
    var: pd.DataFrame
) -> pd.DataFrame:
    """
    Vectorized version of extract_cna_segments (faster).
    """
    segment_list = []

    for w in windows:
        chrom = w['chrom']
        gene_inds = w['genes']
        gene_names = var.index[gene_inds]

        start = var.loc[gene_names, 'start'].min()
        end   = var.loc[gene_names, 'end'].max()

        if chrom not in calls_chr.columns:
            continue

        subset = calls_chr[chrom]
        mask = subset != 0

        seg = pd.DataFrame({
            'cell': subset.index[mask],
            'chromosome': chrom,
            'start': start,
            'end': end,
            'genotype': subset[mask].map({1: 'gain', -1: 'loss'})
        })

        segment_list.append(seg)

    return pd.concat(segment_list, ignore_index=True)


In [12]:
def find_cluster_references(
    adata: sc.AnnData,
    cluster_key: str = 'leiden_clusters',
    layer: str | None = None,
    n_genes_subset: int = 3000,
    gmm_components: int = 2,
    random_state: int = 42,
    verbose: bool = False
) -> dict[str, list[str]]:
    """
    For each cluster in adata.obs[cluster_key], select a diploid reference
    via low‐variance cells. Returns { cluster: [cell, ...], ... }.
    """
    if cluster_key not in adata.obs:
        raise KeyError(f"{cluster_key!r} not in adata.obs")

    X = adata.layers[layer] if layer else adata.X
    if scipy.sparse.issparse(X):
        X = X.tocsr()

    nonmt = ~adata.var_names.str.upper().str.startswith('MT-')
    avail = np.where(nonmt)[0]
    if len(avail) >= n_genes_subset:
        rng = np.random.RandomState(random_state)
        subset_idx = rng.choice(avail, n_genes_subset, replace=False)
    else:
        subset_idx = avail if len(avail)>0 else np.arange(adata.n_vars)

    refs: dict[str, list[str]] = {}

    clusters = [str(x) for x in adata.obs[cluster_key].unique()]

    for cl in clusters:
        mask_cl = (adata.obs[cluster_key].astype(str).values == cl)
        idx_cl  = np.where(mask_cl)[0]
        if len(idx_cl) < 5:
            if verbose:
                print(f"Cluster {cl!r}: only {len(idx_cl)} cells → skipping")
            refs[cl] = []
            continue

        block = X[idx_cl[:, None], subset_idx]

        if scipy.sparse.issparse(block):
            block = block.toarray()

        vars_ = np.nanvar(block, axis=1, ddof=1).reshape(-1,1)

        gmm = GaussianMixture(n_components=gmm_components,
                              random_state=random_state)
        gmm.fit(vars_)
        lbl = gmm.predict(vars_)
        means = gmm.means_.flatten()
        dip = int(np.argmin(means))

        dip_idx   = idx_cl[lbl == dip]
        dip_cells = adata.obs_names[dip_idx].tolist()
        refs[cl]   = dip_cells

        if verbose:
            print(f"Cluster {cl!r}: {len(dip_cells)}/{len(idx_cl)} refs "
                  f"  (GMM means var={means.round(4)})")

    return refs

Run cluster reference function

In [13]:
cluster_refs = find_cluster_references(
    adata,
    cluster_key='leiden_clusters',
    n_genes_subset=3000,
    gmm_components=2,
    random_state=42,
    verbose=True
)


Cluster '14': 6/545 refs   (GMM means var=[0.0793 0.284 ])
Cluster '9': 581/1069 refs   (GMM means var=[0.2005 0.2315])
Cluster '1': 1190/1525 refs   (GMM means var=[0.2025 0.2333])
Cluster '2': 1239/1508 refs   (GMM means var=[0.2334 0.2376])
Cluster '0': 1293/1541 refs   (GMM means var=[0.2242 0.2206])
Cluster '5': 344/1255 refs   (GMM means var=[0.2168 0.2273])
Cluster '6': 1049/1244 refs   (GMM means var=[0.2267 0.2324])
Cluster '10': 611/824 refs   (GMM means var=[0.211  0.2274])
Cluster '15': 201/442 refs   (GMM means var=[0.2341 0.2191])
Cluster '3': 945/1473 refs   (GMM means var=[0.2098 0.1972])
Cluster '4': 1283/1457 refs   (GMM means var=[0.2063 0.2134])
Cluster '12': 504/697 refs   (GMM means var=[0.2174 0.2558])
Cluster '11': 642/786 refs   (GMM means var=[0.2166 0.2243])
Cluster '8': 990/1144 refs   (GMM means var=[0.2256 0.2134])
Cluster '7': 1018/1228 refs   (GMM means var=[0.2184 0.2113])
Cluster '13': 468/566 refs   (GMM means var=[0.2095 0.2334])
Cluster '16': 28/94 

Run detect CNA function

In [14]:
calls_chr, windows, var_sorted = detect_cnas_per_cluster(
    adata,
    cluster_key='leiden_clusters',
    cluster_refs=cluster_refs,
    window_size=100,
    min_genes_per_window=10,
    smoothing_sigma=1.0,
    lfc_threshold=0.05,
    fraction_threshold=0.2,
    layer=None
)

Run segment CNA function

In [15]:
segments_df = extract_cna_segments_fast(
    calls_chr=calls_chr,
    windows=windows,
    var=adata.var
)


In [16]:
cna_segments = extract_cna_segments_fast(calls_chr, windows, var_sorted)

In [17]:
cna_segments = (
    cna_segments
    .merge(
        adata.obs[['leiden_clusters']],
        left_on='cell',
        right_index=True
    )
)
cna_segments.head()

,cell,chromosome,start,end,genotype,leiden_clusters
0,AAACAAGCAACCGGATATCATGTG-1,1,1001138.0,112715332.0,loss,9
1,AAACAAGCAACGACTAATCATGTG-1,1,1001138.0,112715332.0,loss,1
2,AAACAAGCACTTAACCATCATGTG-1,1,1001138.0,112715332.0,loss,9
3,AAACAAGCAGCAATTGATCATGTG-1,1,1001138.0,112715332.0,loss,2
4,AAACAAGCATCAGGACATCATGTG-1,1,1001138.0,112715332.0,loss,2


Summarize CNAs on each chromosome

In [18]:
import pandas as pd

def summarize_chr_cna(segments_df: pd.DataFrame) -> pd.DataFrame:
    summary = (
        segments_df.groupby(['chromosome', 'genotype'])
        .size()
        .unstack(fill_value=0)
        .rename_axis(None, axis=1)
        .reset_index()
    )

    # make up all the columns
    if 'gain' not in summary.columns:
        summary['gain'] = 0
    if 'loss' not in summary.columns:
        summary['loss'] = 0

    return summary.sort_values('chromosome')

In [19]:
summary_df = summarize_chr_cna(segments_df)
display(summary_df)

,chromosome,gain,loss
0,1,141680,438655
1,10,56732,155753
2,11,91894,271194
3,12,74466,221346
4,13,31296,61026
5,14,39180,126380
6,15,38680,127840
7,16,60508,168042
8,17,78714,214938
9,18,24225,56570
